In [ ]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import argparse, imghdr, pickle, posixpath, re
import signal, socket, threading
import urllib.parse, urllib.request
import datetime, os, sys, logging, hashlib
from pathlib import Path
from os import listdir
from os.path import isfile, join

In [ ]:
!unzip Gecko-Binary-Classifier-main.zip

In [ ]:
xform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Note - originally I had the datasets on my drive so that I wouldn't have to upload the datasets constantly. I have left the code below if you would like to run
# these tests on your own using google drive. For more information on running the binary classifier, see the "Training and Testing" section on the README for more details.
# from google.colab import drive
# drive.mount('/content/gdrive')
# dataset_full = datasets.ImageFolder('/content/gdrive/MyDrive/dataset', transform=xform)

dataset_full = datasets.ImageFolder('/content/Gecko-Binary-Classifier-main/dataset', transform=xform)

In [ ]:
n_all = len(dataset_full)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(2000)
dataset_train, dataset_test = torch.utils.data.random_split(dataset_full, [n_train, n_test], rng)
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=4, shuffle=True)

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
torch.nn.init.xavier_uniform_(model.fc.weight)
device = torch.device('cuda:0')
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()

def run_test(model):
  nsamples_test = len(dataset_test)
  loss, correct = 0, 0
  model.eval()
  with torch.no_grad():
    for samples, labels in loader_test:
      samples = samples.to(device)
      labels = labels.to(device)
      outs = model(samples)
      loss += criterion(outs, labels)
      _, preds = torch.max(outs.detach(), 1)
      correct_mask = preds == labels
      correct += correct_mask.sum(0).item()
  return loss / nsamples_test, correct / nsamples_test

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
def run_train(model, opt, sched):
  nsamples_train = len(dataset_train)
  loss_sofar, correct_sofar = 0, 0
  model.train()
  with torch.enable_grad():
    for samples, labels in loader_train:
      samples = samples.to(device)
      labels = labels.to(device)
      opt.zero_grad()
      outs = model(samples)
      _, preds = torch.max(outs.detach(), 1)
      loss = criterion(outs, labels)
      loss.backward()
      opt.step()
      loss_sofar += loss.item() * samples.size(0)
      correct_sofar += torch.sum(preds == labels.detach())
  sched.step()
  return loss_sofar / nsamples_train, correct_sofar / nsamples_train

def run_all(model, optimizer, scheduler, n_epochs):
  for epoch in range(n_epochs):
    loss_train, acc_train = run_train(model, optimizer, scheduler)
    loss_test, acc_test = run_test(model)
    print(f"epoch {epoch}: train loss {loss_train:.4f} acc { acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")

In [ ]:
run_all(model, optimizer, scheduler, 8)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch 0: train loss 0.7293 acc 0.7101, test loss 0.1906 acc 0.7500
epoch 1: train loss 0.6124 acc 0.7918, test loss 0.0826 acc 0.8878
epoch 2: train loss 0.4553 acc 0.8352, test loss 0.1271 acc 0.7449
epoch 3: train loss 0.2588 acc 0.8953, test loss 0.1109 acc 0.8776
epoch 4: train loss 0.4067 acc 0.8621, test loss 0.0624 acc 0.9184
epoch 5: train loss 0.2364 acc 0.9170, test loss 0.0461 acc 0.9337
epoch 6: train loss 0.1715 acc 0.9361, test loss 0.0351 acc 0.9490
epoch 7: train loss 0.2256 acc 0.9042, test loss 0.0316 acc 0.9592
